conceptual checklist:
* what is stemming?
* what is lemmatization?
* when would you want to keep stop words?
* explain word embeddings.
* how do you evaluate an NLP model? 
    * recall
    * precision
    * F1
    * statistical significance?
        * explain particulars
* whether to use supervised or unsupervised?
    * do you already know you're categories?
    * are you familiar with the domain?
* why is BERT so good?
    * transformers
    * attention
* how do you reduce dimensionality of textual data?
    * PCA
    * LDA

Contrived NLP example:

* read in raw text files 
    * jane austen's books
* tokenize
    * words
    * sentences
* clean
    * remove punctuation
    * remove numbers
    * remove stopwords (from pre-baked list)
    * use regex for something
* create features
    * unigrams
    * bigrams
    * lexicon-based
* pass through classification model
* calculating important metrics
    * most frequent words
    * tf-idf
    * most mentioned entities
    * simple sentiment analysis